In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator
from scipy.stats import *

0it [00:00, ?it/s]
/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [58]:
key = pd.read_csv('./key_trunc.csv')

In [59]:
sub = pd.read_csv("./sample_submission_1.csv")

In [335]:
train_melt = pd.read_csv("train_feats.csv")

In [234]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

KeyboardInterrupt: 

In [276]:
def get_median(row, last_n_days):
    return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).median()[-60:])

In [277]:
def get_mean(row, last_n_days):
    return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).mean()[-60:])

In [284]:
def get_std(row, last_n_days):
    return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).std()[-60:])

In [285]:
def get_min(row, last_n_days):
    return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).min()[-60:])

In [303]:
def get_max(row, last_n_days):
    return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).max()[-60:])

In [304]:
def get_mean_overall(row):
    return pd.Series(pd.Series(row)[1:].mean()).repeat(60)

In [297]:
def get_median_overall(row):
    return pd.Series(pd.Series(row)[1:].median()).repeat(60)

In [88]:
# def get_mean_diff(row, last_n_days):
#     return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [89]:
# def get_last_day_visits(row):
#     return row[1:]

In [306]:
def get_long_stats(func_type, df, df_melt, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.apply(get_median, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.apply(get_mean, last_n_days = last_n_days, axis = 1)
    
    elif func_type == "std":
        rolling_stats = df.apply(get_std, last_n_days = last_n_days, axis = 1)

    elif func_type == "min":
        rolling_stats = df.apply(get_min, last_n_days = last_n_days, axis = 1)

    elif func_type == "max":
        rolling_stats = df.apply(get_max, last_n_days = last_n_days, axis = 1)

    elif func_type == "mean_overall":
        rolling_stats = df.apply(get_mean_overall, axis = 1)

    elif func_type == "median_overall":
        rolling_stats = df.apply(get_median_overall, axis = 1)

#     elif func_type == "mean_diff":
#         rolling_stats = df.iloc[:, -61:].progress_apply(get_mean_diff, last_n_days = last_n_days, axis = 1)
        
#     elif func_type == "last_day_visits":        
#         rolling_stats = df.iloc[:, -61:].progress_apply(get_last_day_visits, axis = 1)
    
    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [342]:
def generate_train_melt(train):
    
#     train_melt = pd.read_csv("train_feats.csv")
        
#     Commented because these features are already generated
    train_melt = pd.melt(pd.concat([train.Page, 
                                    train.iloc[:, -60:]],
                                    axis = 1),
                                    id_vars=['Page'], 
                                    var_name="date", value_name="visits")

#     train_melt = get_long_stats("mean", train, train_melt, 7)
    train_melt = get_long_stats("mean", train, train_melt, 30)
    train_melt = get_long_stats("mean", train, train_melt, 60)
    print train_melt.head()

#     train_melt = get_long_stats("median", train, train_melt, 7)
    train_melt = get_long_stats("median", train, train_melt, 30)
    train_melt = get_long_stats("median", train, train_melt, 60)
    print train_melt.head()

    train_melt = get_long_stats("std", train, train_melt, 30)
    train_melt = get_long_stats("std", train, train_melt, 60)
    print train_melt.head()

    train_melt = get_long_stats("min", train, train_melt, 30)
    train_melt = get_long_stats("min", train, train_melt, 60)
    print train_melt.head()

    train_melt = get_long_stats("max", train, train_melt, 30)
    train_melt = get_long_stats("max", train, train_melt, 60)
    print train_melt.head()

    train_melt = get_long_stats("mean_overall", train, train_melt, 60)
    train_melt = get_long_stats("median_overall", train, train_melt, 60)
    print train_melt.head()
    
#     train_melt = get_long_stats("mean_diff", train, train_melt, 7)
#     train_melt = get_long_stats("last_day_visits", train, train_melt)
    
    train_melt["date_num"] = train_melt.apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.apply(get_weekday, axis = 1)
    
    # Columns i am not using
    # train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)

    return train_melt 

def get_lang_features(train_melt):
    
    train_melt = pd.merge(train_melt, pd.concat([train.Page, train.progress_apply(get_language, axis = 1)], axis = 1), on = "Page")
    train_melt.columns = ['Page', 'date', 'visits', 'mean_30', 'mean_60', 'median_30', 'median_60', 'std_30', 'std_60', 'min_30', 'min_60', 'max_30', 'max_60', 'mean_overall', 'median_overall', 'date_num', 'weekday', 'language']
    
    lang_date_melt = train_melt[["language", "date", "mean_30", "mean_60", "median_30", "median_60", "min_30", "min_60", "max_30", "max_60"]]
    
    lang_feats = pd.DataFrame(lang_date_melt.groupby(["language", "date"], as_index = False).aggregate(np.mean))

    lang_feats.columns = ["language", "date", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean", "lang_min_30_mean", "lang_min_60_mean", "lang_max_30_mean", "lang_max_60_mean"]
        
    train_melt = pd.merge(train_melt, lang_feats, on = ["language", "date"])
    
    return train_melt

def get_weekday_features(train_melt):
    
    weekday_page_melt = train_melt[["weekday", "Page", "visits"]]
    
    weekday_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.mean))

    weekday_feats.columns = ["weekday", "Page", "weekday_mean_visits"]
        
    train_melt = pd.merge(train_melt, weekday_feats, on = ["weekday", "Page"])
    
    return train_melt
    
def get_lang_mean(train_melt):
    
    language_page_melt = train_melt[["language", "date", "visits"]]
    
    language_feats = pd.DataFrame(language_page_melt.groupby(["language", "date"], as_index = False).aggregate(np.mean))

    language_feats.columns = ["language", "date", "lang_mean_visits"]
        
    train_melt = pd.merge(train_melt, language_feats, on = ["language", "date"])
    
    return train_melt

train_melt = generate_train_melt(train)
train_melt = get_lang_features(train_melt)
train_melt = get_weekday_features(train_melt)
train_melt = get_lang_mean(train_melt)
train_melt.to_csv("train_feats.csv", index = False)

In [93]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [94]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [95]:
def get_month(row):
    return row.date.split("-")[1]

In [96]:
def get_date(row):
    return row.date.split("-")[2]

In [97]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [193]:
def get_median_test(row, last_index, last_n_days, pred_index):
    
    start_loc = max(min(last_index - last_n_days, pred_index), 0)
    end_loc = min(last_index, pred_index)

    if start_loc < end_loc:
        return np.median(row.iloc[ start_loc : end_loc ].tolist())
    else:
        return np.median(row.iloc[ pred_index - last_n_days : pred_index ].tolist())


In [194]:
def get_mean_test(row, last_index, last_n_days, pred_index):
    
    start_loc = max(min(last_index - last_n_days, pred_index), 0)
    end_loc = min(last_index, pred_index)
    
    if start_loc < end_loc:
        return np.mean(row.iloc[start_loc : end_loc].tolist())
    else:
        return np.mean(row.iloc[pred_index - last_n_days : pred_index].tolist())

In [147]:
# def get_mean_diff_test(row, last_index):
    
#     start_loc = min(last_index - 7, 61)
#     end_loc = min(last_index, 61)
    
#     if start_loc < end_loc:
#         return np.mean([int(x) for x in np.diff( row.iloc[start_loc : end_loc] ) ], dtype=None)
#     else:
#         return np.nan

In [66]:
def get_lang_features_test(test_df):
    
    test_df = pd.merge(test_df, pd.concat([overall_df.Page, overall_df.progress_apply(get_language, axis = 1)], axis = 1), on = "Page")
    test_df.columns = ['Page', 'date', 'visits', 'mean_30', 'mean_60', 'median_30', 'median_60', 'std_30', 'std_60', 'min_30', 'min_60', 'max_30', 'max_60', 'mean_overall', 'median_overall', 'date_num', 'weekday', 'language']
    
    lang_date_melt = test_df[["language", "date", "mean_30", "mean_60", "median_30", "median_60", "min_30", "min_60", "max_30", "max_60"]]
    
    lang_feats = pd.DataFrame(lang_date_melt.groupby(["language", "date"], as_index = False).aggregate(np.mean))
    lang_feats.columns = ["language", "date", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean", "lang_min_30_mean", "lang_min_60_mean", "lang_max_30_mean", "lang_max_60_mean"]
        
    test_df = pd.merge(test_df, lang_feats, on = ["language", "date"])
    
    return test_df

In [148]:
# def get_last_day_visits_test(row, last_index):
    
#     return row.iloc[last_index - 1]

In [219]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [330]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.05
    params['max_depth'] = 7
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0
    
    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)

    return bst

In [331]:
# label_train_melt = np.log1p(train_melt["visits"])
label_train_melt = train_melt["visits"]
# label_train_melt = boxcox(np.array(label_train_melt), lmbda=0.094) #lambda found from default boxcox on outlier-stripped data

# label_train_melt = [0] + np.diff(train_melt["visits"]).tolist()

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("language", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

# train_melt.drop("last_day_visits_None", axis = 1, inplace = True)
# train_melt.drop("mean_diff_7", axis = 1, inplace = True)

# train_melt.drop("mean_7", axis = 1, inplace = True)
# train_melt.drop("median_7", axis = 1, inplace = True)

train_melt.date_num = pd.to_numeric(train_melt.date_num)

In [332]:
x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
bst = run_xgb(x_train, label_train, x_valid, label_valid)

# bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:76381.1	validation-rmse:88278.1	train-smape:166.216	validation-smape:165.32
Multiple eval metrics have been passed: 'validation-smape' will be used for early stopping.

Will train until validation-smape hasn't improved in 10 rounds.
[10]	train-rmse:61039.6	validation-rmse:72275.1	train-smape:78.0295	validation-smape:78.101
[20]	train-rmse:54321.9	validation-rmse:65381.5	train-smape:54.3909	validation-smape:54.7832
[30]	train-rmse:51557.1	validation-rmse:62627.4	train-smape:47.9096	validation-smape:48.2181
[40]	train-rmse:50412.2	validation-rmse:61556.9	train-smape:46.634	validation-smape:46.7683
[50]	train-rmse:49917.6	validation-rmse:61235.7	train-smape:46.5998	validation-smape:46.9658
Stopping. Best iteration:
[42]	train-rmse:50279.3	validation-rmse:61488.9	train-smape:46.4348	validation-smape:46.5786



In [333]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

                feature    fscore
0     lang_mean_30_mean  0.011474
1      lang_min_60_mean  0.013196
2      lang_min_30_mean  0.013483
3   lang_median_30_mean  0.013769
4                min_60  0.014056
5   lang_median_60_mean  0.016638
6      lang_max_30_mean  0.022088
7               mean_60  0.025818
8             median_60  0.026678
9               weekday  0.030120
10     lang_max_60_mean  0.030981
11       median_overall  0.031842
12    lang_mean_60_mean  0.034423
13               min_30  0.041595
14               std_60  0.042456
15             date_num  0.044177
16               max_60  0.046758
17               std_30  0.053069
18         mean_overall  0.059667
19            median_30  0.074871
20              mean_30  0.098394
21               max_30  0.126219
22  weekday_mean_visits  0.128227


In [245]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

# x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
# # x_train = x_train.fillna(0)
# # x_valid = x_valid.fillna(0)
# reg.fit(x_train, label_train)

reg.fit(train_melt, label_train_melt)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [243]:
# prediction = reg.predict(x_valid)
prediction = reg.predict(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean"]])

In [244]:
y_true = np.array(label_valid)
y_pred = np.array(prediction)

denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
diff = np.abs(y_true - y_pred) / denominator
diff[denominator == 0] = 0.0

print np.nanmean(diff)

105.106730814


In [229]:
def get_preds(test_df):
    
    d_test = xgb.DMatrix(test_df)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [55]:
def get_overall_df():
    
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = 0

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [56]:
def get_test_df(overall_df):
    
    test_df = pd.DataFrame()

    pred_index = overall_df.columns.tolist().index('2017-01-01')
    
    for i in range(pred_index, len(overall_df.columns)):

        last_date = overall_df.columns.tolist()[i]
        print last_date
        last_index = i

        test_feats = pd.DataFrame()
        
        test_feats["Page"] = overall_df.Page
        test_feats["date"] = last_date
        
#         test_feats["mean_7"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 7, pred_index = pred_index)
        test_feats["mean_30"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 30, pred_index = pred_index)
        test_feats["mean_60"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 60, pred_index = pred_index)

#         test_feats["median_7"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 7, pred_index = pred_index)
        test_feats["median_30"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 30, pred_index = pred_index)
        test_feats["median_60"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 60, pred_index = pred_index)

#         test_feats["mean_diff_7"] = overall_df.apply(get_mean_diff_test, axis = 1, last_index = last_index)
#         test_feats["last_day_visits_None"] = overall_df.apply(get_last_day_visits_test, axis = 1, last_index = last_index)

#         test_feats["month"] = last_date.split("-")[1]
        test_feats["date_num"] = pd.to_numeric(last_date.split("-")[2])
        test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()
                                
        test_df = pd.concat([test_df, test_feats])
        
    return test_df

In [73]:
overall_df = get_overall_df()
test_df = get_test_df(overall_df)
test_df = get_lang_features_test(test_df)
test_df.to_csv("test_feats.csv", index = False)

In [230]:
test_df = pd.read_csv("test_feats.csv")

In [231]:
prediction = get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean"]])

# prediction = invboxcox(get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday"]]), ld = 0.094)

In [240]:
# pred_index = overall_df.columns.tolist().index("2017-01-01")

# pred_df = overall_df.iloc[:, pred_index:]
# pred_df["Page"] = overall_df.Page

# pred_df_melt = pd.melt(pred_df, id_vars=['Page'], var_name="date", value_name="Visits")        

test_df["Visits"] = prediction
# pred_df = pd.concat([test_df[["Page", "date"]], prediction], axis = 1)
pred_df = test_df[["Page", "date", "Visits"]]

pred_id_visits = pd.merge(pred_df, key, left_on=["Page", "date"], right_on=["trunc_page", "date"], how="inner")[["Id", "Visits"]]

pred_id_visits.to_csv("pred_glm_3.csv", index = False)

In [7]:
from scipy.stats import *
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt if i > 4 and i < 1871])) #Outlier 10% to 90% only
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt]))

bc_label = boxcox(np.array(label_train_melt), lmbda=0.094)

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/stats/morestats.py:516: RuntimeWarning: divide by zero encountered in log
  y = where(lmbda == 0, log(x), (x**lmbda - 1)/lmbda)


In [8]:
def invboxcox(y,ld):
    if ld == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(ld*y+1)/ld))

In [10]:
# np.percentile(label_train_melt, 90)
# np.percentile(label_train_melt, 10)
# len([i for i in label_train_melt if i > 4 and i < 1871])

# import matplotlib.pyplot as plt
# plt.hist(bc_label.tolist())
# plt.show()

# bc_label

# import scipy.special.inv_boxcox
invboxcox(bc_label, ld = 0.094)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


array([ 18.,  11.,   3., ...,   0.,   0.,   0.])

In [341]:
train_melt.head()

,Page,date,visits,mean_30,mean_60,median_30,median_60,std_30,std_60,min_30,...,lang_mean_30_mean,lang_mean_60_mean,lang_median_30_mean,lang_median_60_mean,lang_min_30_mean,lang_min_60_mean,lang_max_30_mean,lang_max_60_mean,weekday_mean_visits,lang_mean_visits
0,2NE1_zh.wikipedia.org_all-access_spider,2016-11-02,18.0,19.000000,21.066667,16.0,17.5,10.218307,13.528209,8.0,...,375.438147,379.927954,315.525245,304.581885,199.268657,166.883804,1055.840427,1658.632113,19.222222,334.657241
1,2PM_zh.wikipedia.org_all-access_spider,2016-11-02,11.0,39.933333,35.983333,26.0,25.5,38.361199,30.086030,11.0,...,375.438147,379.927954,315.525245,304.581885,199.268657,166.883804,1055.840427,1658.632113,29.111111,334.657241
2,3C_zh.wikipedia.org_all-access_spider,2016-11-02,3.0,4.066667,4.983333,4.0,4.0,2.242741,3.270265,0.0,...,375.438147,379.927954,315.525245,304.581885,199.268657,166.883804,1055.840427,1658.632113,27.111111,334.657241
3,4minute_zh.wikipedia.org_all-access_spider,2016-11-02,12.0,19.066667,19.233333,14.0,14.5,18.339581,17.188256,4.0,...,375.438147,379.927954,315.525245,304.581885,199.268657,166.883804,1055.840427,1658.632113,16.888889,334.657241
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,2016-11-02,5.0,13.833333,10.350000,7.0,6.0,30.440539,22.998029,2.0,...,375.438147,379.927954,315.525245,304.581885,199.268657,166.883804,1055.840427,1658.632113,10.888889,334.657241
